# Datago Quiz 

- Editor: Tiger Li 李昌伦
- Date: Jun 15 2019
- 环境: Python 3.6, Jupyter Notebook

In [1]:
# 导入需要的包
import os, re, requests, time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
from multiprocessing import  Pool
from functools import partial

# 文本爬取

## 爬虫应对JS动态数据
发现： 不能采用静态修改Base_URL的方式实现翻页爬虫，因为表格数据来自于javascript脚本，网页URL并不会发生变化

方法： 使用selenium包提供的Chrome浏览器驱动，实现页面切换并用beautifulsoup解析页面html信息，最终转化为csv文件

关于selenium介绍以及安装请点击[此处](https://selenium-python.readthedocs.io/)

**selenium提供了显性/隐性等待策略，能够面对网络状况波动时，加载网页快慢的情况**，具体如何设置参考官方文档链接


In [2]:
# 初始化变量
Base_URL = "http://data.eastmoney.com/report/hyyb.html#dHA9MCZjZz0wJmR0PTQmcGFnZT0x"
PageLimit = 5
current_page = 1

# 自定义数据存储容器： topic：行业名称， title：标题，内容处为 pdf文档下载地址
raw_table = {"topic":[],"title":[],"PDF_url":[]}

# selenium chrome
browser = webdriver.Chrome()
browser.implicitly_wait(30)  # 隐性等待，最长等30秒
browser.get(Base_URL)
locator = (By.ID,"PageCont") 

try:
    while current_page <= PageLimit:
        soup = bs(browser.page_source, "html.parser")
        table = soup.find('tbody')
        # table infos on each page
        for tr in table.find_all('tr'):
            a = tr.find_all('a')
            raw_table['topic'].append(a[0].text)
            raw_table['title'].append(a[5].get('title'))
            raw_table['PDF_url'].append(a[5].get('href'))
        
        # 每10页输出一次进度
        if (current_page%10==0):
            print(" 已完成爬取页面 %d!" % (current_page))
        # 退出while循环
        if current_page == PageLimit:
            break
        # 下一页
        current_page += 1
           
        # 设置显性等待
        browser.find_element_by_xpath("//a[contains(text(),'下一页')]").click()
        WebDriverWait(browser, 20).until(EC.presence_of_element_located(locator))
        time.sleep(5)
        
finally:
    browser.close()

利用Panda Dataframe查看爬取的结果

In [3]:
df = pd.DataFrame(raw_table)
df.head()

,topic,title,PDF_url
0,通讯行业,"创新与拼搏奠定中国设备商全球领先之路‚无惧波折、""中华信""借力5G继续进阶","/report/20190616/hy,APPJ6k1bLdVQIndustry.html"
1,农牧饲渔,农林牧渔行业点评报告：三重因素助推种植走高‚2019农业板块种养齐涨,"/report/20190616/hy,APPJ6kFacP7CIndustry.html"
2,食品饮料,软饮料行业报告三部曲之一：瓶装水：润物无声‚立体透视水的生意经,"/report/20190616/hy,APPJ6kFacP7DIndustry.html"
3,食品饮料,食品饮料行业点评：非洲猪瘟疫苗最新动态点评,"/report/20190616/hy,APPJ6kFacPS7Industry.html"
4,食品饮料,农业&食品周报：猪价近期小幅震荡‚产能维持加速下滑,"/report/20190616/hy,APPJ6kFacPS8Industry.html"


In [33]:
df.to_csv('raw_table.csv', index=False)

## 转换超链接为PDF下载地址
**此小节与下一小节 都可以利用multiprocessing进行加速处理，本质是对dataframe.apply函数的优化** [参考链接](https://stackoverflow.com/questions/26784164/pandas-multiprocessing-apply)

最终我们获得了100页，共计5000条待处理的URL，下面的`get_pdf_url`函数会负责转换url为pdf下载地址，再次用到了requests+beautifulsoup解析

CPU bound 提速明显

In [20]:
# 使用多进程加速
def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

#### 以下两个函数 将会使用apply函数，并且利用multiprocessing加速 ###
# 1. 定义函数用于 获取表格中每一行报告对应的pdf文件链接
def get_pdf_url(x): 
    r = requests.get("http://data.eastmoney.com" + x.PDF_url)
    soup = bs(r.content, "html.parser",from_encoding="iso-8859-1")
    check = soup.select('a[href^="http://pdf"]')
    # 若找不到pdf链接，check会是空list
    if check:
        pdf_url = check[0].get('href')
        return pdf_url
    else:
        # 输出无效pdf链接的行记录
        print("no valid pdf_url at Row:",x.name)
        return 0

# 2. 定义函数用于下载PDF文件
def download_pdf(x):
    # 创建行业对应文件夹
    if x.pdf_url: 
        directory = "dataset/" + x.topic
        if not os.path.exists(directory):
            os.makedirs(directory)
        # 部分文件名中包含/，会造成写入路径错误
        path = directory + "/%s.pdf" % (x.title.replace("/","_"))     
        
        if not os.path.exists(path):
            doc = requests.get(x.pdf_url)   
            with open(path, 'wb') as f:
                f.write(doc.content)
    else:
        print("can't download PDF for invalid URL at Row: %d"%(x.name))

In [3]:
#df = pd.read_csv('raw_table.csv') 
# 测试样本1
test1 = df[4100:4200]
# 以下两块代码为速度测试

In [4]:
%time cache1 = test1.apply(get_pdf_url, axis=1)

no valid pdf_url at Row: 4178
CPU times: user 4.85 s, sys: 79.1 ms, total: 4.93 s
Wall time: 25.5 s


In [5]:
%time cache2 = parallelize_on_rows(test1, get_pdf_url)

no valid pdf_url at Row: 4178
CPU times: user 23.6 ms, sys: 30 ms, total: 53.6 ms
Wall time: 2.49 s


In [9]:
# 需要一段时间
%time cache3 = parallelize_on_rows(df, get_pdf_url)

no valid pdf_url at Row: 4419
no valid pdf_url at Row: 909
no valid pdf_url at Row: 4705
no valid pdf_url at Row: 3579
no valid pdf_url at Row: 3593
no valid pdf_url at Row: 4767
no valid pdf_url at Row: 2952
no valid pdf_url at Row: 3668
no valid pdf_url at Row: 1176
no valid pdf_url at Row: 4178
no valid pdf_url at Row: 1206
no valid pdf_url at Row: 4314
no valid pdf_url at Row: 4369
CPU times: user 275 ms, sys: 105 ms, total: 379 ms
Wall time: 6min 18s


**所以有13条记录无法获取有效的PDF链接**

In [10]:
df['pdf_url'] = cache3
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
topic      5000 non-null object
title      5000 non-null object
PDF_url    5000 non-null object
pdf_url    5000 non-null object
dtypes: object(4)
memory usage: 156.3+ KB


In [11]:
df_new = df.drop(columns="PDF_url")
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
topic      5000 non-null object
title      5000 non-null object
pdf_url    5000 non-null object
dtypes: object(3)
memory usage: 117.3+ KB


## 下载PDF文件

考虑到最终会有100页，共5000份PDF文件需要下载，所以利用multiprocessing对df.apply函数加速

下面两段代码分别是 加速以及不加速的测试， I/O bound所以提速可能不明显

In [17]:
#df_new = pd.read_csv('output.csv')
# 测试样本2
test2 = df_new[1150:1250]

In [18]:
%time cache1 = test2.apply(download_pdf, axis=1)

can't download PDF for invalid URL at Row: 1176
can't download PDF for invalid URL at Row: 1206
CPU times: user 1.75 s, sys: 778 ms, total: 2.53 s
Wall time: 3min 56s


In [19]:
%time cache2 = parallelize_on_rows(test2, download_pdf) 

can't download PDF for invalid URL at Row: 1176
can't download PDF for invalid URL at Row: 1206
CPU times: user 67.6 ms, sys: 47.4 ms, total: 115 ms
Wall time: 1min 1s


In [23]:
#需要一段时间
%time cache3 = parallelize_on_rows(df_new, download_pdf) 

can't download PDF for invalid URL at Row: 4419
can't download PDF for invalid URL at Row: 909
can't download PDF for invalid URL at Row: 2952
can't download PDF for invalid URL at Row: 1176
can't download PDF for invalid URL at Row: 1206
can't download PDF for invalid URL at Row: 4705
can't download PDF for invalid URL at Row: 4178
can't download PDF for invalid URL at Row: 3579
can't download PDF for invalid URL at Row: 3593
can't download PDF for invalid URL at Row: 4767
can't download PDF for invalid URL at Row: 4178
can't download PDF for invalid URL at Row: 3668
can't download PDF for invalid URL at Row: 4314
can't download PDF for invalid URL at Row: 4369
CPU times: user 2.05 s, sys: 876 ms, total: 2.93 s
Wall time: 20min 51s


## 检查结果并存储csv文件

In [30]:
num = df_new[df_new.pdf_url !=0].shape[0]
print("共计下载了 %d份 PDF文件"% num )
df_new.to_csv("result.csv",index=False)

共计下载了 4987份 PDF文件


## 最终成型图
<img src="img/img2.png" width="50%" height="50%">

# 文本分类

## 文本数据预处理
1. 这里使用了[pdfminer3k](pdfminer: https://pypi.org/project/pdfminer3k/)帮助提取文本信息
2. 生成了对应行业名称下的文本文档（.txt文件）
3. 结果存放在了一个新的文件夹`text_data`

In [1]:
#导入需要的包
import re,os
import logging
import jieba 
import numpy as np
import pandas as pd

from io import StringIO
from io import open

# 需要预先安装包 
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, process_pdf

In [4]:
### 文本预处理函数
def pdf2text(path):
    # 输入： PDF文件所在的路径
    # 初始化
    password = ''
    pagenos = set()
    maxpages = 0
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    
    #读取PDF文档
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    with open(path, 'rb') as file:
        process_pdf(rsrcmgr, device, file, pagenos, maxpages=maxpages, password=password, check_extractable=True)

    device.close()
    content = retstr.getvalue()
    retstr.close()
    
    # 去除所有半角全角符号、字母、数字，只留中文。
    lines = str(content).split('\n')
    text = ''.join(lines)
    rule = re.compile(r"[^\u4e00-\u9fa5]")
    text = rule.sub('',text)
    # 输出纯中文文本 字符串
    return text

In [5]:
# 生成所有行业标签
labels = [f for f in os.listdir('dataset/') if not f.startswith('.')] # 避开.DS_store隐藏文件
print(str(len(labels))+"类: "+ '/'.join(labels))

46类: 输配电气/有色金属/公用事业/木业家具/船舶制造/电力行业/文化传媒/软件服务/汽车行业/煤炭采选/文教休闲/商业百货/电子元件/造纸印刷/多元金融/专用设备/食品饮料/电子信息/包装材料/家电行业/玻璃陶瓷/石油行业/医药制造/农牧饲渔/银行/机械行业/房地产/通讯行业/水泥建材/航天航空/旅游酒店/交运物流/装修装饰/交运设备/金属制品/保险/券商信托/工艺商品/环保工程/钢铁行业/纺织服装/综合行业/工程建设/材料行业/化工行业/医疗行业


In [7]:
# ignore warnings
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

error_pdf = 0

for label in labels:
    # 基于每一类制作 词典
    #if label == "农牧饲渔": # 中途暂停，已获得的小规模数据集
    #    break
    #label="木业家具"
    #print("%s 行业开始制作!"% label)
    pdf_dict = []
    sub_dir = [f for f in os.listdir('dataset/'+ label) if not f.startswith('.')]
    for pdf in sub_dir:
        
        # 写入到txt文件中， 涉及I/O 所以耗时较长
        directory = 'txt_data/'+label
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        text_path = directory +"/"+ pdf[:-4]+ '.txt'
        if not os.path.exists(text_path):
            try: 
                pdf_path = 'dataset/'+label +"/"+ pdf
                text = pdf2text(pdf_path)
                with open(text_path, 'w') as f:
                    f.write(text)
            # Exception Handler, 跳过此份问题PDF的读取
            except Exception as e:
                error_pdf += 1
                #print(e)
                continue
                
    print("%s 行业完成!"% label)

print("共计%d PDF文档转换异常!" % error_pdf)

输配电气 行业完成!
有色金属 行业完成!
公用事业 行业完成!
木业家具 行业完成!
船舶制造 行业完成!
电力行业 行业完成!
文化传媒 行业完成!
软件服务 行业完成!
汽车行业 行业完成!
煤炭采选 行业完成!
文教休闲 行业完成!
商业百货 行业完成!
电子元件 行业完成!
造纸印刷 行业完成!
多元金融 行业完成!
专用设备 行业完成!
食品饮料 行业完成!
电子信息 行业完成!
包装材料 行业完成!
家电行业 行业完成!
玻璃陶瓷 行业完成!
石油行业 行业完成!
医药制造 行业完成!
农牧饲渔 行业完成!
银行 行业完成!
机械行业 行业完成!
房地产 行业完成!
通讯行业 行业完成!
水泥建材 行业完成!
航天航空 行业完成!
旅游酒店 行业完成!
交运物流 行业完成!
装修装饰 行业完成!
交运设备 行业完成!
金属制品 行业完成!
保险 行业完成!
券商信托 行业完成!
工艺商品 行业完成!
环保工程 行业完成!
钢铁行业 行业完成!
纺织服装 行业完成!
综合行业 行业完成!
工程建设 行业完成!
材料行业 行业完成!
化工行业 行业完成!
医疗行业 行业完成!
共计345 PDF文档转换异常!


In [8]:
! ls txt_data/* | wc -l

    4604


## 效果图

最终转化得到的文档总数 4604份

<img src="img/img3.png" width="50%" height="50%">



## 读取新数据集
读取数据集，完成分词并存储到dataframe中,此处包含中文分词处理

In [4]:
%%time
frames = []
categories =  [f for f in os.listdir('txt_data') if not f.startswith('.')]

for cate in categories:
    docs = [f for f in os.listdir('txt_data/'+ cate) if not f.startswith('.')]
    txt_list = []
    for doc in docs:
        path = 'txt_data/'+ cate + "/" + doc
        with open(path, "rb") as f:
            text = f.read()
            token = jieba.cut(text)
            new_text = " ".join(token)
            txt_list.append(new_text)
    #print(doc)
    df_tmp = pd.DataFrame({"label": cate, "text": txt_list})
    frames.append(df_tmp)


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/09/xvpww0g933qc13phh23w26mm0000gn/T/jieba.cache
Loading model cost 0.616 seconds.
Prefix dict has been built succesfully.


CPU times: user 2min 46s, sys: 1.37 s, total: 2min 47s
Wall time: 2min 49s


In [5]:
df_all = pd.concat(frames, ignore_index=True)
df_all.to_csv("label_text.csv", index=False)
df_all.label.value_counts()

电子信息    301
汽车行业    285
文化传媒    280
输配电气    277
医药制造    236
通讯行业    217
食品饮料    211
化工行业    202
机械行业    183
电子元件    168
房地产     160
航天航空    148
银行      143
券商信托    133
有色金属    126
钢铁行业    118
商业百货    114
造纸印刷    107
旅游酒店    104
公用事业     98
纺织服装     94
交运物流     94
环保工程     93
家电行业     91
农牧饲渔     89
水泥建材     70
煤炭采选     61
石油行业     56
电力行业     48
工程建设     39
装修装饰     31
多元金融     30
保险       30
材料行业     22
文教休闲     19
交运设备      7
医疗行业      7
金属制品      5
木业家具      4
玻璃陶瓷      2
综合行业      2
软件服务      2
船舶制造      2
专用设备      2
工艺商品      1
包装材料      1
Name: label, dtype: int64

**10折验证要求 由于测试集会被分组，所以每组至少要有一些数据，所以需要提前处理**


In [6]:
df = df_all.groupby("label").filter(lambda x: len(x) > 20)
df.label.value_counts()

电子信息    301
汽车行业    285
文化传媒    280
输配电气    277
医药制造    236
通讯行业    217
食品饮料    211
化工行业    202
机械行业    183
电子元件    168
房地产     160
航天航空    148
银行      143
券商信托    133
有色金属    126
钢铁行业    118
商业百货    114
造纸印刷    107
旅游酒店    104
公用事业     98
纺织服装     94
交运物流     94
环保工程     93
家电行业     91
农牧饲渔     89
水泥建材     70
煤炭采选     61
石油行业     56
电力行业     48
工程建设     39
装修装饰     31
保险       30
多元金融     30
材料行业     22
Name: label, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4459 entries, 0 to 4505
Data columns (total 2 columns):
label    4459 non-null object
text     4459 non-null object
dtypes: object(2)
memory usage: 104.5+ KB


In [2]:
# 导入scikit-learn系列
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import cross_val_score

## 模型搭建与测试

分散训练集与测试集, 考虑到每一个类下有不同数目的文档，设置`stratify`参数,依据标签y，按原数据y中各类比例，分配给train和test，使得train和test中各类数据的比例与原数据集一样。 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], random_state=42, test_size=0.2, stratify=df['label'])

### 特征工程 ###
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# 准备用于 模型预测 输入
X_test_counts = count_vect.transform(X_test)

### 朴素贝叶斯模型预测

In [8]:
%time clf_NB = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf_NB.predict(X_test_counts) 
print("Accuracy : {:.4f}".format(accuracy_score(y_test, y_pred)))

CPU times: user 285 ms, sys: 45.5 ms, total: 330 ms
Wall time: 329 ms
Accuracy : 0.4361


### 逻辑回归模型预测

In [9]:
%time clf_LR = LogisticRegression().fit(X_train_tfidf, y_train)
y_pred = clf_LR.predict(X_test_counts) 
print("Accuracy : {:.4f}".format(accuracy_score(y_test, y_pred)))

CPU times: user 15.4 s, sys: 52.1 ms, total: 15.5 s
Wall time: 15.5 s
Accuracy : 0.8733


### SVM模型预测

In [10]:
%time clf_SVM = LinearSVC().fit(X_train_tfidf, y_train)
y_pred = clf_SVM.predict(X_test_counts) 
print("Accuracy : {:.4f}".format(accuracy_score(y_test, y_pred)))

CPU times: user 4.26 s, sys: 35.6 ms, total: 4.29 s
Wall time: 4.29 s
Accuracy : 0.8924


### 最高精度模型报告

In [11]:
y_pred = clf_SVM.predict(X_test_counts) 
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       交运物流       1.00      0.68      0.81        19
         保险       0.86      1.00      0.92         6
       公用事业       0.69      0.55      0.61        20
       农牧饲渔       1.00      0.94      0.97        18
       券商信托       0.84      0.96      0.90        27
       化工行业       0.92      0.88      0.90        40
       医药制造       1.00      0.98      0.99        47
       商业百货       0.90      0.83      0.86        23
       多元金融       0.00      0.00      0.00         6
       家电行业       1.00      1.00      1.00        18
       工程建设       0.67      0.50      0.57         8
        房地产       1.00      0.97      0.98        32
       文化传媒       0.96      0.98      0.97        56
       旅游酒店       0.91      0.95      0.93        21
       有色金属       1.00      0.96      0.98        25
       机械行业       0.88      0.97      0.92        37
       材料行业       0.00      0.00      0.00         4
       水泥建材       0.87      0.93      0.90   

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## K折-交叉验证

K=10， 所实现的分类模型在 10-fold cross-validation 中的平均 F1 Score

根据之前的三种Accuracy，选择最佳模型：`clf_SVM`

In [12]:
%%time
scores = cross_val_score(clf_SVM, count_vect.transform(df.text), df.label, cv=10, scoring='f1_macro', n_jobs=4)
print("F1 Score 平均: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 Score 平均: 0.78 (+/- 0.02)
CPU times: user 10.1 s, sys: 128 ms, total: 10.2 s
Wall time: 1min 15s


# 结束